# 국민청원 데이터 크롤링을 통한 청원 결과 예측

### selenium을 통해 청원 데이터 크롤링

In [1]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings(action='ignore')

import time
from tqdm import tqdm

# 전처리
import pandas as pd

In [17]:
# 옵션 생성
options = webdriver.ChromeOptions()
# 창 숨기는 옵션 추가
options.add_argument("headless")

# 크롬 버전에 따라 드라이버를 업데이트해주는 ChromeDriverManager
chrome_driver = ChromeDriverManager().install()
service = Service(chrome_driver)

# ChromeDriverManager를 사용하여 더 편하게 selenium실행 가능
driver = webdriver.Chrome(chrome_driver, options=options, service = service)

# 페이지 로드 기다리기X
caps = DesiredCapabilities().CHROME
caps["pageLoadStrategy"] = "none"

### 상태에 따라 청원의 url을 불러와줌

In [3]:
# 성립, 미성립 청원 페이지로 이동
URL = 'https://petitions.assembly.go.kr'
def move_page(URL, type_num):
    if type_num == '성립':
        num = 2
    elif type_num == '미성립':
        num = 3
    
    # 홈페이지로 이동
    driver.get(URL)
    time.sleep(3)
    
    # 팝업 제외 버튼 클릭
    driver.implicitly_wait(10)
    driver.find_element(By.XPATH, r'//*[@id="btnClose"]').click()

    # 동의 종료 청원 클릭
    driver.implicitly_wait(10)
    driver.find_element(By.XPATH, r'//*[@id="mainMenu_list"]/li[3]').click()

    # 성립 청원 버튼 클릭
    driver.implicitly_wait(10)
    driver.find_element(By.XPATH, f'//*[@id="navsubbar"]/ul/li[{num}]').click()

### 성립 청원

In [4]:
# 페이지별로 url을 크롤링하는 함수 생성
def crawl_url(lst):
    # parsing
    html = driver.page_source
    soup = bs(html, 'html.parser')
    # 청원 페이지 크롤링
    contents = soup.select('.BoxWrap')

    for content in contents:
        lst.append(content.find('a')["href"])
        
    time.sleep(1)

In [5]:
# 다음 페이지를 클릭하는 함수 생성
def click_next(page_num, page_type):
    if page_type == '성립':
        click_page = page_num+2

        driver.find_element(By.XPATH, f'//*[@id="contentsbody"]/div/div/div[4]/div[3]/ul/li[{click_page}]/a').click()
        time.sleep(1)
    elif page_type == '미성립':
        if page_num < 6:
            click_page = page_num+2

            driver.find_element(By.XPATH, f'//*[@id="contentsbody"]/div/div/div[4]/div[3]/ul/li[{click_page}]/a').click()
            time.sleep(1)
            
        elif 6 <= page_num <= 61:
            click_page = 8

            driver.find_element(By.XPATH, f'//*[@id="contentsbody"]/div/div/div[4]/div[3]/ul/li[{click_page}]/a').click()
            time.sleep(1)
        
        elif page_num == 62:
            click_page = 9
            driver.find_element(By.XPATH, f'//*[@id="contentsbody"]/div/div/div[4]/div[3]/ul/li[{click_page}]/a').click()
            
        else: # 강제종료
            click_page = 100
            driver.find_element(By.XPATH, f'//*[@id="contentsbody"]/div/div/div[4]/div[3]/ul/li[{click_page}]/a').click()

In [6]:
# 차례로 모든 
def crawler(lst, page_type):
    # 끝까지 크롤링 해줌
    p_n = 1
    while True:
        try:
            crawl_url(lst)
            click_next(p_n, page_type)
            p_n += 1
        except:
            break

In [4]:
# 이동
move_page(URL,'성립')

In [8]:
# 성립 url 크롤링
est_urls = []
crawler(est_urls, '성립')

- 각 url의 데이터 수집

In [9]:
title = []
likes = []
propose = []
contents = []
types = []

In [10]:
for url in tqdm(est_urls):
    est_url = URL + url
    
    # 홈페이지로 이동
    driver.get(est_url)
    time.sleep(1)

    # parsing
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # 제목
    title.append(soup.select_one('h4[title]').text.strip())
    
    # 카테고리
    types.append(soup.find("span", {"class": "title"}, text="청원분야").find_next_sibling("div").text.strip())
    
    # 동의자수
    likes.append(int(soup.select_one('p.hidden').next_sibling.strip().replace(',','').replace('명','')))

    # 청원 취지
    propose.append(soup.select_one('dd.pre').text.strip())

    # 리뷰 내용
    contents.append(soup.select_one('dd.pre.contentTxt').text.strip())

100%|██████████████████████████████████████████████████████████████████████████████████| 51/51 [01:04<00:00,  1.27s/it]


In [11]:
# 데이터로 병합해줌
est_df = pd.DataFrame({'title':title, 'contents': contents, 'like':likes})

### 미성립 청원

In [18]:
move_page(URL, '미성립')

In [8]:
# 미성립 url 크롤링
in_urls = []
crawler(in_urls, '미성립')

- 각 url의 데이터 수집

In [19]:
title = []
likes = []
propose = []
contents = []
types = []

In [20]:
for url_in in in_urls:
    in_url = URL + url_in

    # 홈페이지로 이동
    driver.get(in_url)
    time.sleep(1)

    try:
        alert = driver.switch_to.alert
        alert.accept() # 경고창의 확인 버튼 클릭
        # alert.dismiss() # 경고창의 취소 버튼 클릭
    except:
        pass

    # parsing
    html = driver.page_source
    soup = bs(html, 'html.parser')

    # 제목
    title.append(soup.select_one('h4[title]').text.strip())

    # 카테고리
    types.append(soup.find("span", {"class": "title"}, text="청원분야").find_next_sibling("div").text.strip())

    # 동의자수
    likes.append(int(soup.select_one('p.hidden').next_sibling.strip().replace(',','').replace('명','')))

    # 청원 취지
    propose.append(soup.select_one('dd.pre').text.strip())

    # 리뷰 내용
    contents.append(soup.select_one('dd.pre.contentTxt').text.strip())

In [22]:
# 데이터로 병합해줌
in_df = pd.DataFrame({'title':title, 'contents': contents, 'like':likes, 'types': types})

In [24]:
# 데이터를 csv로 저장
in_df.to_csv('../data/non_established.csv',index = False)
est_df.to_csv('../data/established.csv',index = False)

In [25]:
df = pd.concat([in_df, est_df])
df.drop_duplicates(inplace = True)

In [26]:
df.head()

,title,contents,like
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원,AI 이미지 생성기의 개발은 콘텐츠 산업을 새로운 혁신을 가져다 줄 큰 잠재력을 가...,50000
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원,저는 얼마전 아동학대로 살해당한 한 아이의 삼촌입니다\r\n아동학대 사건의 형량 상...,50000
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원,정부에서 지역별 중도금 가산금리에 대한 차등을 두는것을 아시는지요?\r\n은행에서는...,50000
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원",자금난으로 인한 민간에 송전시장을 민영화보다 차라리 전기요금을 올리세요!\r\n\r...,50000
4,12년간 당한 학교폭력에 관한 청원,"저는 OO초등학교 2003년 입학, OO초등학교에서 2005년 3학년 무렵 OOOO...",50000


In [27]:
df.to_csv('../data/Petition_data.csv', index = False)

#### 전처리 후 코드 csv 파일로 저장

In [ ]:
def remove_newline(x):
    x = x.replace('\n', ' ')
    x = re.sub('\r', '', x)
    x = re.sub('\n', '', x)
    return x

In [3]:
df['content'] = df['contents'].apply(remove_newline)
df['content'] = df['title'] + ' ' + df['content']
df = df.loc[:, ['content', 'like']]
df.head()

,content,like
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,50000
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,50000
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,50000
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",50000
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",50000


In [5]:
df['label'] = np.where(df['like'] >= 10000, 1, 0)
df = df.loc[:, ['content', 'label']]
df.head()

,content,label
0,AI 이미지 생성기의 무분별한 사용과 악용을 막기 위한 법적 규제에 관한 청원 AI...,1
1,아동학대살인 가해자의 엄벌과 신상공개에 관한 청원 저는 얼마전 아동학대로 살해당한 ...,1
2,중도금 가산금리 인하 및 시스템 개편에 관한 청원 정부에서 지역별 중도금 가산금리에...,1
3,"한전은 공기업, 송전시장 민영화 반대에 관한 청원 자금난으로 인한 민간에 송전시장을...",1
4,"12년간 당한 학교폭력에 관한 청원 저는 OO초등학교 2003년 입학, OO초등학교...",1


In [6]:
df.to_csv('../data/Petition_data_advised.csv', index = False)

----------------